Assignment 7 Oscars Movie: Part 2- Gladiators of Dune

In [58]:
!pip install opencv-python


In [149]:
from PIL import Image as pili, ImageOps as piliops
from IPython.display import Image, display
import numpy as np
import pandas as pd
from PIL import Image
import cv2
from PIL import ImageDraw as pild, ImageFont as pilf

Creating Data frame for easy access to Images and Dialogue

In [150]:
# Here we have taken the dialogues as well as image Location 
dialog = pd.read_csv("comic_dialogue_image_mapping.csv", encoding="ISO-8859-1")
dialog

Image Location  \
0        Comic Project/images/IMG1.png   
1        Comic Project/images/IMG2.png   
2        Comic Project/images/IMG3.png   
3        Comic Project/images/IMG4.png   
4        Comic Project/images/IMG6.png   
5        Comic Project/images/IMG5.png   
6        Comic Project/images/IMG7.png   
7       Comic Project/images/IMG16.png   
8       Comic Project/images/IMG13.png   
9       Comic Project/images/IMG11.png   
10      Comic Project/images/IMG12.png   
11      Comic Project/images/IMG15.png   
12      Comic Project/images/IMG18.png   
13      Comic Project/images/IMG17.png   
14       Comic Project/images/IMG3.png   
15      Comic Project/images/IMG21.png   
16      Comic Project/images/IMG22.png   
17      Comic Project/images/IMG16.png   
18      Comic Project/images/IMG23.png   
19      Comic Project/images/IMG25.png   
20      Comic Project/images/IMG26.png   
21      Comic Project/images/IMG27.png   
22      Comic Project/images/IMG28.png   
23      Comic Project/images/IMG29.png   
24      Comic Project/images/IMG30.png   
25      Comic Project/images/IMG31.png   
26      Comic Project/images/IMG32.png   
27      Comic Project/images/IMG33.png   
28      Comic Project/images/IMG34.png   
29  Comic Project/images/IMG-BG-31.png   
30      Comic Project/images/IMG36.png   
31      Comic Project/images/IMG37.png   
32      Comic Project/images/IMG38.png   
33      Comic Project/images/IMG39.png   
34      Comic Project/images/IMG40.png   
35      Comic Project/images/IMG41.png   
36      Comic Project/images/IMG42.png   
37      Comic Project/images/IMG43.png   
38      Comic Project/images/IMG44.png   
39      Comic Project/images/IMG46.png   
40      Comic Project/images/IMG47.png   
41      Comic Project/images/IMG48.png   

                                            Dialogues  
0     Lucius Atreides: "I will see this empire fall."  
1   Acacius Harkonnen: "Your fate was sealed the m...  
2   Emperor Caracalla IV: "You dared challenge my ...  
3   Chani Valeria: "Lucius
 I won't let them have ...  
4          Lucius Atreides: "What treachery is this?"  
5   Acacius Harkonnen: "You will kneel before me, ...  
6              Emperor Caracalla IV: "Take him away!"  
7   Chani Valeria: "I'll get him out of here, no m...  
8   Lucius Atreides: "They think I am broken. But ...  
9   Lucius Atreides: "They throw me into this cage...  
10  Emperor Caracalla IV: "Let them see how weak y...  
11   Acacius Harkonnen: "Your end is near, Atreides."  
12   Chani Valeria: "Fight, Lucius. You can do this."  
13  Lucius Atreides: "I see it now
 the path to vi...  
14       Emperor Caracalla IV: "Its over, Atreides."  
15      Acacius Harkonnen: "You have nowhere to run!"  
16  Lucius Atreides: "Your tech wont save you
 on...  
17         Chani Valeria: "I cant let him die here."  
18             Fremen warrior: "We fight as one now."  
19        Lucius Atreides: "Thank you for saving me."  
20         Emperor Caracalla IV: "No! Not like this!"  
21  Chani Valeria: "The way out is this way. Follo...  
22  Lucius Atreides: "I cant trust anyone, but yo...  
23  Acacius Harkonnen: "Find them! They cant esca...  
24  Lucius Atreides: "Were almost there. Just a l...  
25         Lucius Atreides: "Why should I trust you?"  
26  Chani Valeria: "Because together, we can defea...  
27       Lucius Atreides: "Then Ill fight with you."  
28    Chani Valeria: "Welcome to the Fremen, Lucius."  
29     Lucius Atreides: "This is only the beginning."  
30  Chani Valeria: "We take the fight to them. No ...  
31         Acacius Harkonnen: "You cannot defeat us."  
32  Emperor Caracalla IV: "This will be your final...  
33                    Lucius Atreides: "For freedom!"  
34                    Chani Valeria: "We strike now!"  
35             Emperor Caracalla IV: "Hold the line!"  
36        Acacius Harkonnen: "You cant win, Lucius!"  
37    Lucius Atreides: "I will make you regret that."  
38                  Chani Valeria

In [152]:
import cv2
import os
from PIL import Image  # Import PIL for image saving

# Function to Compress Image by Reducing its Dimensions Here, we use the resize function from openCV to resize all the images to a standard size.
def imgcompress_mem(path_in, k):
    img = cv2.imread(path_in, cv2.IMREAD_UNCHANGED)

    # if condition to check if the image is loaded correctly
    if img is None:
        print(f"Error: Could not load image - {path_in}")
        return None  # Return None to handle errors gracefully

    # Setting the ratio of resized image
    width = 451  # int((img.shape[1]) / k)
    height = 351  # int((img.shape[0]) / k)

    # Resizing the image using OpenCV
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

# Controls blur amount and line size Using the medianBlur function, we cartoonize the images to make them look like a comic.
def cartoonizebl_mem(path_in, k, blur, line):
    imgc = imgcompress_mem(path_in, k)

    # if condition to ensure the image was loaded successfully
    if imgc is None:
        return None

    line_size = line
    blur_value = blur

    # Convert to grayscale
    gray = cv2.cvtColor(imgc, cv2.COLOR_BGR2GRAY)

    # Apply median blur
    gray_blur = cv2.medianBlur(gray, blur_value)

    # Apply adaptive thresholding to get edges
    bigedges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)

    # Create a cartoon effect by masking the edges
    return cv2.bitwise_and(imgc, imgc, mask=bigedges)

# Function to Convert Images to Cartoonized Versions and Save the Output We loop through the dialogues dataframe to convert the images and save the cartoonized images to a different folder.
def convert_images(x):
    image_path = os.path.join(f"{x}")

    # if condition to check if file exists before processing
    if not os.path.exists(image_path):
        print(f"Error: File not found - {image_path}")
        return

    # Apply the cartoonize effect
    cblimg = cartoonizebl_mem(image_path, 5.1, 3, 3)

    # if condition to ensure cartoonization was successful
    if cblimg is None:
        print(f"Skipping: {image_path} (cartoonization failed)")
        return

    # Convert OpenCV BGR image to RGB for PIL
    cblimg_pil = cv2.cvtColor(cblimg, cv2.COLOR_BGR2RGB)

    # Convert NumPy array to PIL image
    pil_image = Image.fromarray(cblimg_pil)

    # Ensure output folder exists
    output_folder = os.path.join("cartoon", os.path.dirname(x))  # Create subdirectories if needed
    os.makedirs(output_folder, exist_ok=True)

    # Define output path
    output_path = os.path.join(output_folder, os.path.basename(x))  # Save the image with the same filename

    # Save the image
    pil_image.save(output_path)
    print(f"Saved cartoonized image: {output_path}")



# Loop Through Image Locations and Apply Cartoonization
for x in dialog["Image Location"]:
    convert_images(x)


Saved cartoonized image: cartoon\Comic Project/images\IMG1.png
Saved cartoonized image: cartoon\Comic Project/images\IMG2.png
Saved cartoonized image: cartoon\Comic Project/images\IMG3.png
Saved cartoonized image: cartoon\Comic Project/images\IMG4.png
Saved cartoonized image: cartoon\Comic Project/images\IMG6.png
Saved cartoonized image: cartoon\Comic Project/images\IMG5.png
Saved cartoonized image: cartoon\Comic Project/images\IMG7.png
Saved cartoonized image: cartoon\Comic Project/images\IMG16.png
Saved cartoonized image: cartoon\Comic Project/images\IMG13.png
Saved cartoonized image: cartoon\Comic Project/images\IMG11.png
Saved cartoonized image: cartoon\Comic Project/images\IMG12.png
Saved cartoonized image: cartoon\Comic Project/images\IMG15.png
Saved cartoonized image: cartoon\Comic Project/images\IMG18.png
Saved cartoonized image: cartoon\Comic Project/images\IMG17.png
Saved cartoonized image: cartoon\Comic Project/images\IMG3.png
Saved cartoonized image: cartoon\Comic Project/i

In [154]:
import cv2
import os
from PIL import Image, ImageDraw, ImageFont  # Import PIL for image saving and drawing text

# Function to Compress Image by Reducing its Dimensions
def imgcompress_mem(path_in, k):
    img = cv2.imread(path_in, cv2.IMREAD_UNCHANGED)

    # if condition to check if the image is loaded correctly
    if img is None:
        print(f"Error: Could not load image - {path_in}")
        return None  # Return None to handle errors gracefully

    # Setting the ratio of resized image
    width = 451  # int((img.shape[1]) / k)
    height = 351  # int((img.shape[0]) / k)

    # Resizing the image using OpenCV
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

# Controls blur amount and line size Using the medianBlur function, we cartoonize the images to make them look like a comic.
def cartoonizebl_mem(path_in, k, blur, line):
    imgc = imgcompress_mem(path_in, k)

    # if condition to ensure the image was loaded successfully
    if imgc is None:
        return None

    line_size = line
    blur_value = blur

    # Convert to grayscale
    gray = cv2.cvtColor(imgc, cv2.COLOR_BGR2GRAY)

    # Apply median blur
    gray_blur = cv2.medianBlur(gray, blur_value)

    # Apply adaptive thresholding to get edges
    bigedges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)

    # Create a cartoon effect by masking the edges
    return cv2.bitwise_and(imgc, imgc, mask=bigedges)

# Function to Convert Images to Cartoonized Versions, Add Border, and Save the Output
def convert_images(image_path, dialog_text):
    # if condition to check if file exists before processing
    if not os.path.exists(image_path):
        print(f"Error: File not found - {image_path}")
        return

    # Apply the cartoonize effect
    cblimg = cartoonizebl_mem(image_path, 5.1, 3, 3)

    # if condition to ensure cartoonization was successful
    if cblimg is None:
        print(f"Skipping: {image_path} (cartoonization failed)")
        return

    # Convert OpenCV BGR image to RGB for PIL
    cblimg_pil = cv2.cvtColor(cblimg, cv2.COLOR_BGR2RGB)

    # Convert NumPy array to PIL image
    pil_image = Image.fromarray(cblimg_pil)

    # Add black border first
    border_thickness = 10
    pil_image_with_border = Image.new('RGB', (pil_image.width + 2 * border_thickness, pil_image.height + 2 * border_thickness), (0, 0, 0))  # Black border
    pil_image_with_border.paste(pil_image, (border_thickness, border_thickness))

    # Add white outline around the black border
    outline_thickness = 4
    final_width = pil_image_with_border.width + 2 * outline_thickness
    final_height = pil_image_with_border.height + 2 * outline_thickness

    pil_image_with_outline = Image.new('RGB', (final_width, final_height), (255, 255, 255))  # White outline
    pil_image_with_outline.paste(pil_image_with_border, (outline_thickness, outline_thickness))

    # Initialize ImageDraw object to add text
    draw = ImageDraw.Draw(pil_image_with_outline)

    # Use a basic font
    try:
        font = ImageFont.truetype("arial.ttf", 14)  # Adjusted to make it smaller
    except IOError:
        font = ImageFont.load_default()

    # Calculate text size and position (bottom of the image)
    image_width, image_height = pil_image_with_outline.size
    text_bbox = draw.textbbox((0, 0), dialog_text, font=font)  # Get the bounding box of the text
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    text_x = (image_width - text_width) / 2  # Center the text horizontally
    text_y = image_height - text_height - 10  # Position text near the bottom

    # Add text at the bottom of the image
    draw.text((text_x, text_y), dialog_text, font=font, fill=(255, 255, 255))  # Set text color to white for contrast

    # Ensure output folder exists
    output_folder = os.path.join("cartoon", os.path.dirname(image_path))  # Create subdirectories if needed
    os.makedirs(output_folder, exist_ok=True)

    # Define output path
    output_path = os.path.join(output_folder, os.path.basename(image_path))  # Save the image with the same filename

    # Save the image with border and text
    pil_image_with_outline.save(output_path)
    print(f"Saved cartoonized image with dialogue and border: {output_path}")

# Data for images and dialogues
data = [
    {"image": "Comic Project/images/IMG1.png", "dialogue": "Lucius Atreides: \"I will see this empire fall.\""},
    {"image": "Comic Project/images/IMG2.png", "dialogue": "Prahlad Narayan (Harkonnen): \"Your fate was sealed the moment you defied me.\""},
    {"image": "Comic Project/images/IMG3.png", "dialogue": "Emperor Caracalla IV: \"You dared challenge my rule, Lucius. Now, you fight for your life!\""},
    {"image": "Comic Project/images/IMG4.png", "dialogue": "Chani Valeria: \"Lucius… I won't let them have you.\""},
    {"image": "Comic Project/images/IMG6.png", "dialogue": "Lucius Atreides: \"What treachery is this?\""},
    {"image": "Comic Project/images/IMG5.png", "dialogue": "Prahlad Narayan (Harkonnen): \"You will kneel before me, or die.\""},
    {"image": "Comic Project/images/IMG7.png", "dialogue": "Emperor Caracalla IV: \"Take him away!\""},
    {"image": "Comic Project/images/IMG16.png", "dialogue": "Chani Valeria: \"I’ll get him out of here, no matter the cost.\""},
    {"image": "Comic Project/images/IMG13.png", "dialogue": "Lucius Atreides: \"They think I am broken. But I have seen the future… and they will fall.\""},
    {"image": "Comic Project/images/IMG11.png", "dialogue": "Lucius Atreides: \"They throw me into this cage… a battle of beasts and machines.\""},
    {"image": "Comic Project/images/IMG12.png", "dialogue": "Emperor Caracalla IV: \"Let them see how weak you are, Lucius.\""},
    {"image": "Comic Project/images/IMG15.png", "dialogue": "Prahlad Narayan (Harkonnen): \"Your end is near, Atreides.\""},
    {"image": "Comic Project/images/IMG18.png", "dialogue": "Chani Valeria: \"Fight, Lucius. You can do this.\""},
    {"image": "Comic Project/images/IMG17.png", "dialogue": "Lucius Atreides: \"I see it now… the path to victory.\""},
    {"image": "Comic Project/images/IMG3.png", "dialogue": "Emperor Caracalla IV: \"It’s over, Atreides.\""},
    {"image": "Comic Project/images/IMG21.png", "dialogue": "Acacius Harkonnen: \"You have nowhere to run!\""},
    {"image": "Comic Project/images/IMG22.png", "dialogue": "Lucius Atreides: \"Your tech won’t save you… only true warriors survive.\""},
    {"image": "Comic Project/images/IMG16.png", "dialogue": "Chani Valeria: \"I can’t let him die here.\""},
    {"image": "Comic Project/images/IMG23.png", "dialogue": "Fremen warrior: \"We fight as one now.\""},
    {"image": "Comic Project/images/IMG25.png", "dialogue": "Lucius Atreides: \"Thank you for saving me.\""},
    {"image": "Comic Project/images/IMG26.png", "dialogue": "Emperor Caracalla IV: \"No! Not like this!\""},
    {"image": "Comic Project/images/IMG27.png", "dialogue": "Chani Valeria: \"The way out is this way. Follow me.\""},
    {"image": "Comic Project/images/IMG28.png", "dialogue": "Lucius Atreides: \"I can’t trust anyone, but you, Chani.\""},
    {"image": "Comic Project/images/IMG29.png", "dialogue": "Acacius Harkonnen: \"Find them! They can’t escape.\""},
    {"image": "Comic Project/images/IMG30.png", "dialogue": "Lucius Atreides: \"We’re almost there. Just a little further.\""},
    {"image": "Comic Project/images/IMG31.png", "dialogue": "Lucius Atreides: \"Why should I trust you?\""},
    {"image": "Comic Project/images/IMG32.png", "dialogue": "Chani Valeria: \"Because together, we can defeat them.\""},
    {"image": "Comic Project/images/IMG33.png", "dialogue": "Lucius Atreides: \"Then I’ll fight with you.\""},
    {"image": "Comic Project/images/IMG34.png", "dialogue": "Chani Valeria: \"Welcome to the Fremen, Lucius.\""},
    {"image": "Comic Project/images/IMG-BG-31.png", "dialogue": "Lucius Atreides: \"This is only the beginning.\""},
    {"image": "Comic Project/images/IMG36.png", "dialogue": "Chani Valeria: \"We take the fight to them. No more hiding.\""},
    {"image": "Comic Project/images/IMG37.png", "dialogue": "Acacius Harkonnen: \"You cannot defeat us.\""},
    {"image": "Comic Project/images/IMG38.png", "dialogue": "Emperor Caracalla IV: \"This will be your final battle.\""},
    {"image": "Comic Project/images/IMG39.png", "dialogue": "Lucius Atreides: \"For freedom!\""},
    {"image": "Comic Project/images/IMG40.png", "dialogue": "Chani Valeria: \"We strike now!\""},
    {"image": "Comic Project/images/IMG41.png", "dialogue": "Emperor Caracalla IV: \"Hold the line!\""},
    {"image": "Comic Project/images/IMG42.png", "dialogue": "Acacius Harkonnen: \"You can’t win, Lucius!\""},
    {"image": "Comic Project/images/IMG43.png", "dialogue": "Lucius Atreides: \"I will make you regret that.\""},
    {"image": "Comic Project/images/IMG44.png", "dialogue": "Chani Valeria: \"You are nothing.\""},
    {"image": "Comic Project/images/IMG46.png", "dialogue": "Acacius Harkonnen: \"You... can’t defeat me.\""},
    {"image": "Comic Project/images/IMG47.png", "dialogue": "Lucius Atreides: \"It’s over, Harkonnen.\""},
    {"image": "Comic Project/images/IMG48.png", "dialogue": "Emperor Caracalla IV: \"No! This isn’t the end!\""}
]


# Process each image with its dialogue
for item in data:
    convert_images(item['image'], item['dialogue'])


Saved cartoonized image with dialogue and border: cartoon\Comic Project/images\IMG1.png
Saved cartoonized image with dialogue and border: cartoon\Comic Project/images\IMG2.png
Saved cartoonized image with dialogue and border: cartoon\Comic Project/images\IMG3.png
Saved cartoonized image with dialogue and border: cartoon\Comic Project/images\IMG4.png
Saved cartoonized image with dialogue and border: cartoon\Comic Project/images\IMG6.png
Saved cartoonized image with dialogue and border: cartoon\Comic Project/images\IMG5.png
Saved cartoonized image with dialogue and border: cartoon\Comic Project/images\IMG7.png
Saved cartoonized image with dialogue and border: cartoon\Comic Project/images\IMG16.png
Saved cartoonized image with dialogue and border: cartoon\Comic Project/images\IMG13.png
Saved cartoonized image with dialogue and border: cartoon\Comic Project/images\IMG11.png
Saved cartoonized image with dialogue and border: cartoon\Comic Project/images\IMG12.png
Saved cartoonized image with

In [155]:
import os
from PIL import Image

def create_comic_page(image_paths, rows, cols):
    # Open images
    images = [Image.open(img_path) for img_path in image_paths]
    
    # Ensure there are enough images to fill the grid
    total_images = rows * cols
    if len(images) < total_images:
        print(f"Warning: Not enough images to fill the grid. Only {len(images)} images available.")
        rows = len(images) // cols  # Adjust rows based on available images
        if len(images) % cols != 0:
            rows += 1  # Add an extra row if there are remaining images
    
    # Resize images to fit into the grid
    panel_width = max(img.width for img in images)  # Width of largest image
    panel_height = max(img.height for img in images)  # Height of largest image

    # Create a new blank image for the comic page
    comic_page_width = panel_width * cols
    comic_page_height = panel_height * rows
    comic_page = Image.new("RGB", (comic_page_width, comic_page_height), (255, 255, 255))

    # Place images on the page
    x_offset = 0
    y_offset = 0
    for i, img in enumerate(images):
        img_resized = img.resize((panel_width, panel_height))
        comic_page.paste(img_resized, (x_offset, y_offset))
        
        x_offset += panel_width
        if (i + 1) % cols == 0:
            x_offset = 0
            y_offset += panel_height

    return comic_page

def images_to_pdf(image_paths, output_pdf_path, rows, cols, poster_image_path):
    pages = []
    images_per_page = rows * cols

    # Add poster as the first page
    poster_image = Image.open(poster_image_path)

    # Get dimensions from the first comic page to resize the poster accordingly
    sample_page = create_comic_page(image_paths[:images_per_page], rows, cols)
    poster_image_resized = poster_image.resize(sample_page.size)

    pages.append(poster_image_resized)

    # Split images into pages based on the grid configuration
    for i in range(0, len(image_paths), images_per_page):
        page_images = image_paths[i:i + images_per_page]
        comic_page = create_comic_page(page_images, rows, cols)
        pages.append(comic_page)
    
    # Create the directory if it doesn't exist
    output_dir = os.path.dirname(output_pdf_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save as PDF
    pages[0].save(output_pdf_path, save_all=True, append_images=pages[1:])

# Path to the poster image
poster_image_path = "Gladiators of Dune Poster.jpeg"  # Change this path to your actual poster image

# Sequence of image paths
image_sequence = [
    "Comic Project/images/IMG1.png",
    "Comic Project/images/IMG2.png",
    "Comic Project/images/IMG3.png",
    "Comic Project/images/IMG4.png",
    "Comic Project/images/IMG6.png",
    "Comic Project/images/IMG5.png",
    "Comic Project/images/IMG7.png",
    "Comic Project/images/IMG16.png",
    "Comic Project/images/IMG13.png",
    "Comic Project/images/IMG11.png",
    "Comic Project/images/IMG12.png",
    "Comic Project/images/IMG15.png",
    "Comic Project/images/IMG18.png",
    "Comic Project/images/IMG17.png",
    "Comic Project/images/IMG3.png",
    "Comic Project/images/IMG21.png",
    "Comic Project/images/IMG22.png",
    "Comic Project/images/IMG16.png",
    "Comic Project/images/IMG23.png",
    "Comic Project/images/IMG25.png",
    "Comic Project/images/IMG26.png",
    "Comic Project/images/IMG27.png",
    "Comic Project/images/IMG28.png",
    "Comic Project/images/IMG29.png",
    "Comic Project/images/IMG30.png",
    "Comic Project/images/IMG31.png",
    "Comic Project/images/IMG32.png",
    "Comic Project/images/IMG33.png",
    "Comic Project/images/IMG34.png",
    "Comic Project/images/IMG-BG-31.png",
    "Comic Project/images/IMG36.png",
    "Comic Project/images/IMG37.png",
    "Comic Project/images/IMG38.png",
    "Comic Project/images/IMG39.png",
    "Comic Project/images/IMG40.png",
    "Comic Project/images/IMG41.png",
    "Comic Project/images/IMG42.png",
    "Comic Project/images/IMG43.png",
    "Comic Project/images/IMG44.png",
    "Comic Project/images/IMG46.png",
    "Comic Project/images/IMG47.png",
    "Comic Project/images/IMG48.png"
]

# Example usage
image_paths = [os.path.join("cartoon", path) for path in image_sequence]

# Set rows and columns for the comic page
rows, cols = 3, 3  # Adjust based on your preference and available images

# Create and save the PDF with the poster
output_pdf_path = "Comic_Project/output/storybook.pdf"
images_to_pdf(image_paths, output_pdf_path, rows, cols, poster_image_path)


In [ ]:
import cv2
import os
from PIL import Image, ImageDraw, ImageFont
from deep_translator import GoogleTranslator

# Function to Compress Image by Reducing its Dimensions
def imgcompress_mem(path_in, k):
    img = cv2.imread(path_in, cv2.IMREAD_UNCHANGED)

    # if condition to check if the image is loaded correctly
    if img is None:
        print(f"Error: Could not load image - {path_in}")
        return None  # Return None to handle errors gracefully

    # Setting the ratio of resized image
    width = 451  # Fixed width
    height = 351  # Fixed height

    # Resizing the image using OpenCV
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

# Controls blur amount and line size Using the medianBlur function, we cartoonize the images to make them look like a comic.
def cartoonizebl_mem(path_in, k, blur, line):
    imgc = imgcompress_mem(path_in, k)

    # if condition to ensure the image was loaded successfully
    if imgc is None:
        return None

    line_size = line
    blur_value = blur

    # Convert to grayscale
    gray = cv2.cvtColor(imgc, cv2.COLOR_BGR2GRAY)

    # Apply median blur
    gray_blur = cv2.medianBlur(gray, blur_value)

    # Apply adaptive thresholding to get edges
    bigedges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)

    # Create a cartoon effect by masking the edges
    return cv2.bitwise_and(imgc, imgc, mask=bigedges)

# Initializing the translator for translating to Hindi
translator = GoogleTranslator(source='en', target='hi')

# Function to translate text
def translate_text(text):
    try:
        # Translate the input text to Hindi
        return translator.translate(text)
    except Exception as e:
        # If an error occurs, return the error message
        print(f"Translation error: {e}")
        return str(e)

# Function to Convert Images to Cartoonized Versions, Add Border, and Save the Output
def convert_images(image_path, dialog_text):
    # Translate the dialog text to Hindi
    translated_dialog = translate_text(dialog_text)

    # if not condition to check if file exists before processing
    if not os.path.exists(image_path):
        print(f"Error: File not found - {image_path}")
        return

    # Applying the cartoonize effect
    cblimg = cartoonizebl_mem(image_path, 5.1, 3, 3)

    # if condition to ensure cartoonization was successful
    if cblimg is None:
        print(f"Skipping: {image_path} (cartoonization failed)")
        return

    # Convert OpenCV BGR image to RGB for PIL
    cblimg_pil = cv2.cvtColor(cblimg, cv2.COLOR_BGR2RGB)

    # Convert NumPy array to PIL image
    pil_image = Image.fromarray(cblimg_pil)

    # Adding Border with black color
    border_thickness = 10
    pil_image_with_border = Image.new('RGB', (pil_image.width + 2 * border_thickness, pil_image.height + 2 * border_thickness), (0, 0, 0))  # Changed (255, 255, 255) to (0, 0, 0) for black
    pil_image_with_border.paste(pil_image, (border_thickness, border_thickness))

    # Adding a white outline around the black border
    outline_thickness = 5
    pil_image_with_outline = Image.new('RGB', (pil_image_with_border.width + 2 * outline_thickness, pil_image_with_border.height + 2 * outline_thickness), (255, 255, 255))  # White outline
    pil_image_with_outline.paste(pil_image_with_border, (outline_thickness, outline_thickness))

    # Initialize ImageDraw object to add text
    draw = ImageDraw.Draw(pil_image_with_outline)

    # Specifying the font path correctly
    font_path = 'Downloads/Noto_Serif_Devanagari/NotoSerifDevanagari-VariableFont_wdth,wght.ttf'  # Ensure this font file exists
    try:
        # Use ImageFont.truetype to load the Hindi font
        font = ImageFont.truetype(font_path, 14)  # Adjust font size if necessary
    except IOError:
        print(f"Font not found at {font_path}, using default font.")
        font = ImageFont.load_default()  # Fallback if font loading fails

    # To calculate text size and position (bottom of the image)
    image_width, image_height = pil_image_with_outline.size
    text_bbox = draw.textbbox((0, 0), translated_dialog, font=font)  # Get the bounding box of the text
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    text_x = (image_width - text_width) / 2  # Center the text horizontally
    text_y = image_height - text_height - 10  # Position text near the bottom

    # Add text at the bottom of the image
    draw.text((text_x, text_y), translated_dialog, font=font, fill=(255, 255, 255))  # Set text color to white for contrast

    # Ensure output folder exists with the correct subdirectory structure
    output_folder = os.path.join("cartoon", "Comic Project_Hindi", "images")  # Fixed path structure
    os.makedirs(output_folder, exist_ok=True)

    # Define output path using the base name of the image
    output_path = os.path.join(output_folder, os.path.basename(image_path))  # Save the image with the same filename

    # Save the image with border, outline, and text
    pil_image_with_outline.save(output_path)
    print(f"Saved cartoonized image with translated dialogue and border: {output_path}")

# Data for images and dialogues (in English)
data = [
  {"image": "Comic Project/images/IMG1.png", "dialogue": "Lucius Atreides: \"I will see this empire fall.\""},
    {"image": "Comic Project/images/IMG2.png", "dialogue": "Acacius Harkonnen: \"Your fate was sealed the moment you stepped into my domain.\""},
    {"image": "Comic Project/images/IMG3.png", "dialogue": "Emperor Caracalla IV: \"You dared challenge my rule?\""},
    {"image": "Comic Project/images/IMG4.png", "dialogue": "Chani Valeria: \"Lucius… I won’t let them have you.\""},
    {"image": "Comic Project/images/IMG6.png", "dialogue": "Lucius Atreides: \"What treachery is this?\""},
    {"image": "Comic Project/images/IMG5.png", "dialogue": "Acacius Harkonnen: \"You will kneel before me, Atreides.\""},
    {"image": "Comic Project/images/IMG7.png", "dialogue": "Emperor Caracalla IV: \"Take him away!\""},
    {"image": "Comic Project/images/IMG16.png", "dialogue": "Chani Valeria: \"I’ll get him out of here, no matter the cost.\""},
    {"image": "Comic Project/images/IMG13.png", "dialogue": "Lucius Atreides: \"They think I am broken. But I will rise again.\""},
    {"image": "Comic Project/images/IMG11.png", "dialogue": "Lucius Atreides: \"They throw me into this cage… but I will break free.\""},
    {"image": "Comic Project/images/IMG12.png", "dialogue": "Emperor Caracalla IV: \"Let them see how weak you truly are.\""},
    {"image": "Comic Project/images/IMG15.png", "dialogue": "Acacius Harkonnen: \"Your end is near, Atreides.\""},
    {"image": "Comic Project/images/IMG18.png", "dialogue": "Chani Valeria: \"Fight, Lucius. You can do this.\""},
    {"image": "Comic Project/images/IMG17.png", "dialogue": "Lucius Atreides: \"I see it now… the path to victory.\""},
    {"image": "Comic Project/images/IMG3.png", "dialogue": "Emperor Caracalla IV: \"It’s over, Atreides.\""},
    {"image": "Comic Project/images/IMG21.png", "dialogue": "Acacius Harkonnen: \"You have nowhere to run!\""},
    {"image": "Comic Project/images/IMG22.png", "dialogue": "Lucius Atreides: \"Your tech won’t save you… on this day.\""},
    {"image": "Comic Project/images/IMG16.png", "dialogue": "Chani Valeria: \"I can’t let him die here.\""},
    {"image": "Comic Project/images/IMG23.png", "dialogue": "Fremen warrior: \"We fight as one now.\""},
    {"image": "Comic Project/images/IMG25.png", "dialogue": "Lucius Atreides: \"Thank you for saving me.\""},
    {"image": "Comic Project/images/IMG26.png", "dialogue": "Emperor Caracalla IV: \"No! Not like this!\""},
    {"image": "Comic Project/images/IMG27.png", "dialogue": "Chani Valeria: \"The way out is this way. Follow me.\""},
    {"image": "Comic Project/images/IMG28.png", "dialogue": "Lucius Atreides: \"I can’t trust anyone, but you… I’ll follow.\""},
    {"image": "Comic Project/images/IMG29.png", "dialogue": "Acacius Harkonnen: \"Find them! They can’t escape.\""},
    {"image": "Comic Project/images/IMG30.png", "dialogue": "Lucius Atreides: \"We’re almost there. Just a little more.\""},
    {"image": "Comic Project/images/IMG31.png", "dialogue": "Lucius Atreides: \"Why should I trust you?\""},
    {"image": "Comic Project/images/IMG32.png", "dialogue": "Chani Valeria: \"Because together, we can defeat them all.\""},
    {"image": "Comic Project/images/IMG33.png", "dialogue": "Lucius Atreides: \"Then I’ll fight with you.\""},
    {"image": "Comic Project/images/IMG34.png", "dialogue": "Chani Valeria: \"Welcome to the Fremen, Lucius.\""},
    {"image": "Comic Project/images/IMG-BG-31.png", "dialogue": "Lucius Atreides: \"This is only the beginning.\""},
    {"image": "Comic Project/images/IMG36.png", "dialogue": "Chani Valeria: \"We take the fight to them. No retreat.\""},
    {"image": "Comic Project/images/IMG37.png", "dialogue": "Acacius Harkonnen: \"You cannot defeat us.\""},
    {"image": "Comic Project/images/IMG38.png", "dialogue": "Emperor Caracalla IV: \"This will be your final stand.\""},
    {"image": "Comic Project/images/IMG39.png", "dialogue": "Lucius Atreides: \"For freedom!\""},
    {"image": "Comic Project/images/IMG40.png", "dialogue": "Chani Valeria: \"We strike now!\""},
    {"image": "Comic Project/images/IMG41.png", "dialogue": "Emperor Caracalla IV: \"Hold the line!\""},
    {"image": "Comic Project/images/IMG42.png", "dialogue": "Acacius Harkonnen: \"You can’t win, Lucius!\""},
    {"image": "Comic Project/images/IMG43.png", "dialogue": "Lucius Atreides: \"I will make you regret that.\""},
    {"image": "Comic Project/images/IMG44.png", "dialogue": "Chani Valeria: \"You are nothing.\""},
    {"image": "Comic Project/images/IMG46.png", "dialogue": "Acacius Harkonnen: \"You… can’t defeat me.\""},
    {"image": "Comic Project/images/IMG47.png", "dialogue": "Lucius Atreides: \"It’s over, Harkonnen.\""},
    {"image": "Comic Project/images/IMG48.png", "dialogue": "Emperor Caracalla IV: \"No! This isn’t the end!\""}
]

# Process each image and dialogue pair
for entry in data:
    convert_images(entry["image"], entry["dialogue"])


Saved cartoonized image with translated dialogue and border: cartoon\Comic Project_Hindi\images\IMG1.png
Saved cartoonized image with translated dialogue and border: cartoon\Comic Project_Hindi\images\IMG2.png
Saved cartoonized image with translated dialogue and border: cartoon\Comic Project_Hindi\images\IMG3.png
Saved cartoonized image with translated dialogue and border: cartoon\Comic Project_Hindi\images\IMG4.png
Saved cartoonized image with translated dialogue and border: cartoon\Comic Project_Hindi\images\IMG6.png
Saved cartoonized image with translated dialogue and border: cartoon\Comic Project_Hindi\images\IMG5.png
Saved cartoonized image with translated dialogue and border: cartoon\Comic Project_Hindi\images\IMG7.png
Saved cartoonized image with translated dialogue and border: cartoon\Comic Project_Hindi\images\IMG16.png
Saved cartoonized image with translated dialogue and border: cartoon\Comic Project_Hindi\images\IMG13.png
Saved cartoonized image with translated dialogue and 

In [145]:
import os
from PIL import Image

def create_comic_page(image_paths, rows, cols):
    # Open images
    images = [Image.open(img_path) for img_path in image_paths]
    
    # To ensure there are enough images to fill the grid
    total_images = rows * cols
    if len(images) < total_images:
        print(f"Warning: Not enough images to fill the grid. Only {len(images)} images available.")
        rows = len(images) // cols  # Adjusting rows based on available images
        if len(images) % cols != 0:
            rows += 1  # Adding an extra row if there are remaining images
    
    # Resizing images to fit into the grid
    panel_width = max(img.width for img in images)  # Width of largest image
    panel_height = max(img.height for img in images)  # Height of largest image

    # To create a new blank image for the comic page
    comic_page_width = panel_width * cols
    comic_page_height = panel_height * rows
    comic_page = Image.new("RGB", (comic_page_width, comic_page_height), (255, 255, 255))

    # Placing images on the page
    x_offset = 0
    y_offset = 0
    for i, img in enumerate(images):
        img_resized = img.resize((panel_width, panel_height))
        comic_page.paste(img_resized, (x_offset, y_offset))
        
        x_offset += panel_width
        if (i + 1) % cols == 0:
            x_offset = 0
            y_offset += panel_height

    return comic_page

def images_to_pdf(image_paths, output_pdf_path, rows, cols, poster_image_path):
    pages = []
    images_per_page = rows * cols

    # Add poster as the first page
    poster_image = Image.open(poster_image_path)

    # Getting dimensions from the first comic page to resize the poster accordingly
    sample_page = create_comic_page(image_paths[:images_per_page], rows, cols)
    poster_image_resized = poster_image.resize(sample_page.size)

    pages.append(poster_image_resized)

    # Splitting images into pages based on the grid configuration
    for i in range(0, len(image_paths), images_per_page):
        page_images = image_paths[i:i + images_per_page]
        comic_page = create_comic_page(page_images, rows, cols)
        pages.append(comic_page)
    
    # Creating the directory if it doesn't exist
    output_dir = os.path.dirname(output_pdf_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save as PDF
    pages[0].save(output_pdf_path, save_all=True, append_images=pages[1:])

# Path to the poster image
poster_image_path = "Gladiators of Dune Poster.jpeg"  # Change this path to your actual poster image

# Sequence of image paths
image_sequence = [
    "Comic Project_Hindi/images/IMG1.png",
    "Comic Project_Hindi/images/IMG2.png",
    "Comic Project_Hindi/images/IMG3.png",
    "Comic Project_Hindi/images/IMG4.png",
    "Comic Project_Hindi/images/IMG6.png",
    "Comic Project_Hindi/images/IMG5.png",
    "Comic Project_Hindi/images/IMG7.png",
    "Comic Project_Hindi/images/IMG16.png",
    "Comic Project_Hindi/images/IMG13.png",
    "Comic Project_Hindi/images/IMG11.png",
    "Comic Project_Hindi/images/IMG12.png",
    "Comic Project_Hindi/images/IMG15.png",
    "Comic Project_Hindi/images/IMG18.png",
    "Comic Project_Hindi/images/IMG17.png",
    "Comic Project_Hindi/images/IMG3.png",
    "Comic Project_Hindi/images/IMG21.png",
    "Comic Project_Hindi/images/IMG22.png",
    "Comic Project_Hindi/images/IMG16.png",
    "Comic Project_Hindi/images/IMG23.png",
    "Comic Project_Hindi/images/IMG25.png",
    "Comic Project_Hindi/images/IMG26.png",
    "Comic Project_Hindi/images/IMG27.png",
    "Comic Project_Hindi/images/IMG28.png",
    "Comic Project_Hindi/images/IMG29.png",
    "Comic Project_Hindi/images/IMG30.png",
    "Comic Project_Hindi/images/IMG31.png",
    "Comic Project_Hindi/images/IMG32.png",
    "Comic Project_Hindi/images/IMG33.png",
    "Comic Project_Hindi/images/IMG34.png",
    "Comic Project_Hindi/images/IMG-BG-31.png",
    "Comic Project_Hindi/images/IMG36.png",
    "Comic Project_Hindi/images/IMG37.png",
    "Comic Project_Hindi/images/IMG38.png",
    "Comic Project_Hindi/images/IMG39.png",
    "Comic Project_Hindi/images/IMG40.png",
    "Comic Project_Hindi/images/IMG41.png",
    "Comic Project_Hindi/images/IMG42.png",
    "Comic Project_Hindi/images/IMG43.png",
    "Comic Project_Hindi/images/IMG44.png",
    "Comic Project_Hindi/images/IMG46.png",
    "Comic Project_Hindi/images/IMG47.png",
    "Comic Project_Hindi/images/IMG48.png"
]
  

# Example usage
image_paths = [os.path.join("cartoon", path) for path in image_sequence]

# Set rows and columns for the comic page
rows, cols = 3, 3  # Adjust based on your preference and available images

# Create and save the PDF with the poster
output_pdf_path = "Comic_Project/output/storybook_Hindi.pdf"
images_to_pdf(image_paths, output_pdf_path, rows, cols, poster_image_path)
